
<img src="Images/ebay_logo.png">
# <center> Collecting eBay Listings Data

<center> This script creates a CSV file of ebay listings with 
     
<center> 15 columns of data to use for analysis.
<br>

## <center> Important: 

<center> This script is made to search sold ebay listings.
    
<center> Make sure you are looking at completed listings and that you do
    
<center>all your search filtering on ebay before you collect your input URL.

In [1]:
# How many results do you want?

desired_number_of_results = 150


# URL for page 1 of listings you want to scrape?

input_link = 'https://www.ebay.com/sch/Small-Cents-/11633/i.html?_from=R40&_sop=1&_nkw=ms67&LH_Sold=1&LH_Complete=1&rt=nc&LH_All=4&_ipg=50&_pgn=1'


# Where Would you like to Save your CSV File?

csv_title = 'Data/Sold_MS67_Cents'


# File will be to saved to the location of this notebook.

## <center> Example of page ebay results page.
<img src="Images/ebay_search.png">
    

 <center> Don't forget to run input cell.
    

# <center> Time To Start Scraping
 <center> Now run cell below to get results!
    

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')



# Prepare inputs

partial_link = input_link[:-1]

d = desired_number_of_results/50
ru = math.ceil(d)

csv_title = f'{csv_title}.csv'

# Loop through ebay listings.

n = 0
ebay_listings = []

for r in range(ru):
    
    n = n+1
    url = f'{partial_link}{str(n)}' 
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('li', class_='s-item')
    
    
    for result in results: 
     
                title = result.find('h3', class_="s-item__title")

                title1 = list(title.strings)[-1]
            
                l_title = len(title1)
            
                status = list(title.strings)[0]

                s_split= status.split(" ")
                sold= s_split[0]


                price = result.find('div', class_="s-item__details").span.text
                
                end_time = result.find('span', class_="s-item__ended-date").text
                time = end_time[7:]
                date = end_time[:6]
                year = status[-4:]
                shipping = result.find('span', class_="s-item__shipping").text
                link = result.a['href']
                
                try:
                    result.find('span', class_="s-item__bids").text                    
                except AttributeError:                   
                    bids= 'BIN'
                    f_type= 'Buy_it_now'
                else:
                    bids = result.find('span', class_="s-item__bids").text 
                    f_type = 'Auction'
                try:
                    result.find('span', class_="x-tooltip").text
                except AttributeError:
                    trp = 'No'
                else:
                    trp ='Yes'                
                try:
                    result.find('span', class_='s-item__free-returns').text
                except AttributeError:
                    returns = 'No'
                else:
                    returns= 'Yes'
                try:
                    result.find('span', class_='s-item__purchase-options s-item__purchaseOptions').text
                except AttributeError:
                    boa= 'No'
                else:
                    boa= 'Yes'

            
               
                
                #Add to list of listings
                ebay_listings.append({'Format': f_type,
                                      'Best_Offer_Accepted': boa,
                                      'Result': sold,                                   
                                      'Title': title1,
                                      'Price': price,
                                      'Bids': bids,
                                      'Shipping': shipping,
                                      'Free_Returns': returns,
                                      'Top_Rated_Plus': trp,
                                      'Sale_Date': date,
                                      'Sale_Time': time,
                                      'Sale_Year': year,
                                      'Link': link, 
                                      'Title_Length': l_title 
                                })
               
                
df = pd.DataFrame(ebay_listings)
df = df.iloc[:desired_number_of_results]

ebay_listings_df = df[['Result','Title','Format','Best_Offer_Accepted','Price','Bids','Shipping','Free_Returns','Top_Rated_Plus','Sale_Time','Sale_Date','Sale_Year','Link','Title_Length']]
# ebay_listings_df.to_csv(csv_title)

ebay_listings_df.head()

,Result,Title,Format,Best_Offer_Accepted,Price,Bids,Shipping,Free_Returns,Top_Rated_Plus,Sale_Time,Sale_Date,Sale_Year,Link,Title_Length
0,Sold,1949-S 1C RD Lincoln Wheat One Cent NGC MS67...,Buy_it_now,Yes,$132.00,BIN,Free Shipping,Yes,No,23:40,Feb-18,2019,https://www.ebay.com/itm/1949-S-1C-RD-Lincoln-...,67
1,Sold,1953-S 1C RD Lincoln Wheat One Cent NGC MS67...,Buy_it_now,Yes,$128.00,BIN,Free Shipping,Yes,No,23:39,Feb-18,2019,https://www.ebay.com/itm/1953-S-1C-RD-Lincoln-...,67
2,Sold,1953-S 1C RD Lincoln Wheat Cent PCGS MS67RD ...,Buy_it_now,Yes,$146.00,BIN,Free Shipping,Yes,No,23:36,Feb-18,2019,https://www.ebay.com/itm/1953-S-1C-RD-Lincoln-...,57
3,Sold,1943-S 1C Lincoln Wheat One Cent PCGS MS67 ...,Buy_it_now,Yes,$248.00,BIN,Free Shipping,Yes,No,23:26,Feb-18,2019,https://www.ebay.com/itm/1943-S-1C-Lincoln-Whe...,55
4,Sold,1 old graded coin USA 1 cent Lincoln Memorial ...,Auction,No,$20.31,10 bids,+$3.19 shipping,No,No,19:51,Feb-18,2019,https://www.ebay.com/itm/1-old-graded-coin-USA...,71


# <center> Data Wrangling
### <center> (Creating More Useful Columns)

In [3]:

df1 = ebay_listings_df

# Refomrat Shipping cost

df1['Shipping']= df1['Shipping'].str.replace('Free Shipping', '0')
df1['Shipping']= df1['Shipping'].str.replace(' shipping', '')
df1['Shipping']= df1['Shipping'].str.replace('$', '')
df1['Shipping']= df1['Shipping'].str.replace('+', '')
df1['Shipping']= df1['Shipping'].astype(float)


# Create Price + Shipping colum 

df1['Price']= df1['Price'].str.replace('$', '')
df1['Price']= df1['Price'].str.replace(',', '')
df1['Price'] = df1['Price'].astype(float)
df1['Price_+_Shipping']= df1['Price']+ df1['Shipping']

# Convert Title to Upper Case

df1['Title'] = df1['Title'].str.upper()

# Create Grader column

df1.loc[df1['Title'].str.contains('PCGS'), 'Grader'] = 'PCGS'
df1.loc[df1['Title'].str.contains('NGC'), 'Grader'] = 'NGC'
df1.loc[df1['Title'].str.contains('ANACS'), 'Grader'] = 'ANACS'
df1.loc[df1['Title'].str.contains('ICG'), 'Grader'] = 'ICG'

# Drop Coins without listed Grader

df1['Length']= df1.Grader.str.len()
df1 = df1[df1.Length < 5 ]


# Create Date column
df1['Coin_Date'] = df1.Title.str.extract('(\d\d\d\d)')

# Crete CAC column

df1.loc[df1['Title'].str.contains(' CAC '), 'CAC'] = 'CAC'


# Create Mint column

df1.loc[df1['Title'].str.contains(' D '), 'Mint'] = 'D'
df1.loc[df1['Title'].str.contains('-D'), 'Mint'] = 'D'
df1.loc[df1['Title'].str.contains(' S '), 'Mint'] = 'S'
df1.loc[df1['Title'].str.contains('-S'), 'Mint'] = 'S'

# Replace NaN Values

df1['Mint'].fillna(' ', inplace=True)
df1['CAC'].fillna(' ', inplace=True)

# Refomat Bids 

df1['Bids']= df1['Bids'].str.replace(' bids', '')
df1['Bids']= df1['Bids'].str.replace(' bid', '')
df1['Bids']= df1['Bids'].str.replace('BIN', '-')

# Create Time Stamp Column

df1['Time_Stamp']= df1['Sale_Date']+ df1['Sale_Year']
df1['Time_Stamp'] = df1['Time_Stamp'].str.replace('201', '-201')
df1['Time_Stamp'] = pd.to_datetime(df1['Time_Stamp'])


# Create day of the week Column

df1['Sale_Day'] = df1['Time_Stamp'].dt.weekday_name


In [4]:
# Organize columns

fdf = df1[[
    'Coin_Date',
     'Mint',
    'Grader',
    'CAC',
    'Price_+_Shipping',
     'Price',
    'Shipping',
     'Bids',
    'Result',
    'Free_Returns',
    'Top_Rated_Plus',
     'Format',
  'Best_Offer_Accepted',
     'Title_Length',
     'Sale_Time',
    'Sale_Day',
     'Sale_Date',
    'Sale_Year',
    'Time_Stamp',
]]
# Columns not included 
# Title, Link


# Save to CSV

fdf.to_csv(csv_title)
fdf

,Coin_Date,Mint,Grader,CAC,Price_+_Shipping,Price,Shipping,Bids,Result,Free_Returns,Top_Rated_Plus,Format,Best_Offer_Accepted,Title_Length,Sale_Time,Sale_Day,Sale_Date,Sale_Year,Time_Stamp
0,1949,S,NGC,,132.00,132.00,0.00,-,Sold,Yes,No,Buy_it_now,Yes,67,23:40,Monday,Feb-18,2019,2019-02-18
1,1953,S,NGC,,128.00,128.00,0.00,-,Sold,Yes,No,Buy_it_now,Yes,67,23:39,Monday,Feb-18,2019,2019-02-18
2,1953,S,PCGS,,146.00,146.00,0.00,-,Sold,Yes,No,Buy_it_now,Yes,57,23:36,Monday,Feb-18,2019,2019-02-18
3,1943,S,PCGS,,248.00,248.00,0.00,-,Sold,Yes,No,Buy_it_now,Yes,55,23:26,Monday,Feb-18,2019,2019-02-18
4,1970,D,ICG,,23.50,20.31,3.19,10,Sold,No,No,Auction,No,71,19:51,Monday,Feb-18,2019,2019-02-18
5,1970,,ICG,,6.19,3.00,3.19,4,Sold,No,No,Auction,No,69,19:48,Monday,Feb-18,2019,2019-02-18
6,1972,,ICG,,31.25,28.06,3.19,7,Sold,No,No,Auction,No,69,19:45,Monday,Feb-18,2019,2019-02-18
7,1943,S,PCGS,,435.00,435.00,0.00,25,Sold,No,No,Auction,No,80,18:16,Monday,Feb-18,2019,2019-02-18
8,1967,,NGC,,17.99,17.99,0.00,1,Sold,Yes,Yes,Auction,No,60,18:01,Monday,Feb-18,2019,2019-02-18
9,2004,D,PCGS,,44.95,44.95,0.00,-,Sold,Yes,Yes,Buy_it_now,No,80,13:21,Monday,Feb-18,2019,2019-02-18


### <center> When you see the output of the  pandas data frame :
<center>Your CSV file has been saved in your choosen Location.